In [1]:
import pyforest

In [2]:
df_path = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/paper/datasets/80train_20test/featurized/'
neks = ['NEK2_binding', 'NEK2_inhibition', 'NEK3_binding', 'NEK5_binding', 'NEK9_binding', 'NEK9_inhibition']
model_types = ['GP', 'RF']
feat_types=['MOE','MFP']
RF_types = ['RF', 'RF_BCW', 'BRFC', 'BRFC_BCW'] 
samplings = ['none_scaled','UNDER','SMOTE']
rf_path = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/paper/results/RF_results/'
gp_path = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/paper/results/GP_results/'
cmpd_df_path = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/paper/results/compound_TNFPFNTP/'
col_order = ['NEK', 'feat_type', 'strategy','modeling_type', 'compound_id', 'active', 'prediction' , 'RF_type']


ideas 
- see overlapping molecules between all test sets
- then see if it's in test/train

In [3]:
all_neks = []
# all_nek_dfs=pd.DataFrame()
for nek in neks: 
    df_orig = pd.read_csv(f'{df_path}{nek}_MFP_none_scaled.csv')
    df_UNDER = pd.read_csv(f'{df_path}{nek}_MFP_UNDER.csv')
    # all_neks.append(df_orig[df_orig['subset']=='test']) 
    # all_neks.append(df_UNDER[df_UNDER['subset']=='test']) 
    
    unique_cmpds = set(df_orig['compound_id']).intersection(set(df_UNDER['compound_id']))
    
    unique_cmpds_nek = df_orig[df_orig['compound_id'].isin(unique_cmpds)]
    unique_cmpds_nek_test=unique_cmpds_nek[unique_cmpds_nek['subset']=='test']
    all_neks.append(unique_cmpds_nek_test)
    

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
def common_unique_values(list1, list2, *lists):
    sets = [set(list1), set(list2)]
    for lst in lists:
        sets.append(set(lst))
    return set.intersection(*sets)


In [29]:
all_dfs = [] 
all_neks = []
for nek in neks:  
    full_df = pd.read_csv(f'{df_path}{nek}_MFP_none_scaled.csv') 
    # full_df=full_df[full_df['subset']=='train']
    all_dfs.append(full_df)
    all_neks.append(nek) 
    
df_dict = {}
for df, nek in zip(all_dfs, all_neks):
    df_dict[nek] = df


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [30]:
neks = ['NEK2_binding', 'NEK2_inhibition', 'NEK3_binding', 'NEK5_binding', 'NEK9_binding', 'NEK9_inhibition']
NEK2_binding=df_dict['NEK2_binding']
NEK2_inhibition=df_dict['NEK2_inhibition']  
NEK3_binding=df_dict['NEK3_binding']
NEK5_binding=df_dict['NEK5_binding']
NEK9_binding=df_dict['NEK9_binding']
NEK9_inhibition=df_dict['NEK9_inhibition']   

In [31]:
nek2inhibtest=NEK2_inhibition[NEK2_inhibition['subset']=='test']
nek9inhibtest=NEK9_inhibition[NEK9_inhibition['subset']=='test']
common_inhib_test=common_unique_values(list(nek2inhibtest['compound_id'].values),list(nek9inhibtest['compound_id'].values))
common_inhib_test

{'kdb_2013',
 'kdb_2212',
 'kdb_2238',
 'kdb_2908',
 'kdb_3160',
 'kdb_3164',
 'kdb_3211',
 'kdb_3224',
 'kdb_3232',
 'kdb_3241',
 'kdb_3264',
 'kdb_3290',
 'kdb_3305',
 'kdb_3330',
 'kdb_3334'}

In [64]:
nek2bindtest=NEK2_binding[NEK2_binding['subset']=='test']
nek9bindtest=NEK9_binding[NEK9_binding['subset']=='test']
nek3bindtest=NEK3_binding[NEK3_binding['subset']=='test']
nek5bindtest=NEK5_binding[NEK5_binding['subset']=='test']
common_binding_test = common_unique_values(list(nek2bindtest['compound_id'].values),list(nek9bindtest['compound_id'].values),
                           list(nek3bindtest['compound_id'].values),list(nek5bindtest['compound_id'].values)     )
common_binding_test

{'kdb_2181'}

In [57]:
nek9_inhib_selects=[]
nek2_inhib_selects=[]

for cmpd in common_inhib_test:
    for nek_df in [NEK2_inhibition, NEK9_inhibition]: 
        nek_val = nek_df['NEK'].values[0]
        match = nek_df[nek_df['compound_id'] == cmpd][['compound_id','active']]
        # print(match)
        if nek_val == 'NEK2_inhibition':
            nek2_inhib_selects.append(match)
        elif nek_val == 'NEK9_inhibition':
            nek9_inhib_selects.append(match)


In [68]:
nek9inhib_df = pd.concat(nek9_inhib_selects).reset_index(drop=True)
nek2inhib_df = pd.concat(nek2_inhib_selects).reset_index(drop=True)
inhibition_selectivity = pd.merge(nek2inhib_df,nek9inhib_df, on='compound_id',
                        how='outer',suffixes=('_NEK2_inhibition','_NEK9_inhibition'))


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [69]:
inhibition_selectivity

,compound_id,active_NEK2_inhibition,active_NEK9_inhibition
0,kdb_3330,0,0
1,kdb_2212,0,0
2,kdb_3211,0,0
3,kdb_3160,0,0
4,kdb_2013,0,0
5,kdb_3164,0,0
6,kdb_3264,1,0
7,kdb_3290,0,0
8,kdb_2908,0,0
9,kdb_3305,0,0


In [72]:
nek9_bind_selects=[]
nek2_bind_selects=[]
nek3_bind_selects=[]
nek5_bind_selects=[]

for cmpd in common_binding_test:
    for nek_df in [NEK2_binding, NEK3_binding,NEK5_binding,NEK9_binding ]: 
        nek_val = nek_df['NEK'].values[0]
        match = nek_df[nek_df['compound_id'] == cmpd][['compound_id','active']]
        # print(match)
        if nek_val == 'NEK2_binding':
            nek2_bind_selects.append(match)
        elif nek_val == 'NEK3_binding':
            nek3_bind_selects.append(match)
        elif nek_val == 'NEK5_binding':
            nek5_bind_selects.append(match)
        elif nek_val == 'NEK9_binding':
            nek9_bind_selects.append(match)
nek9bind_df = pd.concat(nek9_bind_selects).reset_index(drop=True)
nek2bind_df = pd.concat(nek2_bind_selects).reset_index(drop=True)
nek3bind_df = pd.concat(nek3_bind_selects).reset_index(drop=True)
nek5bind_df = pd.concat(nek5_bind_selects).reset_index(drop=True)
nek3bind_df = nek3bind_df.rename(columns={'active':'active_NEK3_binding'})
nek5bind_df = nek5bind_df.rename(columns={'active':'active_NEK5_binding'})
binding_selectivity = pd.merge(nek2bind_df,nek9bind_df, on='compound_id',
                               how='outer',suffixes=('_NEK2_binding', '_NEK9_binding'))

binding_selectivity = pd.merge(binding_selectivity, nek5bind_df, on='compound_id', how='outer')
binding_selectivity = pd.merge(binding_selectivity, nek3bind_df, on='compound_id', how='outer')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [73]:
binding_selectivity

,compound_id,active_NEK2_binding,active_NEK9_binding,active_NEK5_binding,active_NEK3_binding
0,kdb_2181,0,0,0,0
